# Microbiota analysis
## Author: Tijs van Lieshout

### Import statements and setting up[ config:

In [74]:
from math import pi
import pandas as pd
import glob
import yaml
from IPython.display import display
from bokeh.plotting import figure
from bokeh.layouts import gridplot
from bokeh.models import Range1d, ColumnDataSource
from bokeh.io import output_notebook, show
from bokeh.transform import dodge
output_notebook()

Loading BokehJS ...

In [75]:
def get_config():
    '''
    Function that gets the configuration of personalized variable definitions
    '''
    with open("config.yaml", 'r') as stream:
        config = yaml.safe_load(stream)
    return config

### Loading the data:

In [76]:
def load_data_microbiota(PATH):
    """Load microbiota data (Gut Feeling Knowledge Base and metaphlann output) into pandas 
    dataframes

    Keyword arguments:
    PATH -- The path which contains the Gut Feeling Knowledge Base and metaphlann output dir
    
    Returns:
    gfkb -- A pandas dataframe containing the Gut Feeling Knowledge Base
    tax_profiles -- A pandas dataframe containing the taxonomic profile per barcode for all barcodes"""
    
    # Gut Feeling Knowledge Base
    gfkb = pd.read_csv(f"{PATH}/GutFeelingKnowledgeBase-v4-Master_List.csv")
    gfkb = gfkb.drop(columns=["Present in GFKB v3 (Y/N)",
                              "Present in GFKB_epilepsy v3 (Y/N)"])
    gfkb = pd.concat([gfkb.drop(columns=["Genome Size (Mb)"]).apply(lambda x: x.astype(str)), 
                      gfkb["Genome Size (Mb)"]], axis=1)

    tax_profiles = pd.DataFrame()
    
    # Dictionary containing barcode ID as key and a list with subject ID and bool if vegan as values
    barcode2subject_sample = {"barcode_01":["A", True],
                              "barcode_02":["B", True],
                              "barcode_03":["C", True],
                              "barcode_04":["D", True],
                              "barcode_05":["E", True],
                              "barcode_06":["A", False],
                              "barcode_07":["B", False],
                              "barcode_08":["C", False],
                              "barcode_09":["D", False],
                              "barcode_10":["E", False]}
    
    # Concatenate all taxonomic profiles of all barcodes to one dataframe
    tax_profiles = concat_tax_profiles(PATH, tax_profiles, barcode2subject_sample)
    
    return gfkb, tax_profiles

In [77]:
def concat_tax_profiles(PATH, tax_profiles, barcode2subject_sample):
    """Concatenate all taxonomic profiles of all given barcodes in the metaphlan output dir
    to one dataframe and reset the index on multi-index (subject, is_vegan).

    Keyword arguments:
    PATH -- The path which contains the Gut Feeling Knowledge Base and metaphlann output dir
    tax_profiles -- An empty pandas dataframe containing the column names in which all taxonomic
    profiles will be concatenated.
    barcode2subject_sample -- Dictionary containing barcode ID as key and a list with subject ID 
    and bool if sample is vegan as values
    
    Returns:
    tax_profiles -- A pandas dataframe containing the taxonomic profile per barcode for all barcodes"""
    
    for file in glob.glob(f"{PATH}/adj_align_output/*.txt"):
        tax_profile = pd.read_csv(file, 
                                  comment="#", 
                                  sep="\t", 
                                  names=["clade_name", 
                                         "NCBI_tax_id", 
                                         "relative_abundance",
                                         "additional_species"])
        # Splitting clade_name into taxonomic levels
        tax_profile = tax_profile.join(tax_profile["clade_name"].str.split('|', expand=True).rename(columns={0:'kingdom', 
                                                                                                             1:'phylum', 
                                                                                                             2:'class', 
                                                                                                             3:'order', 
                                                                                                             4:'family', 
                                                                                                             5:'genus', 
                                                                                                             6:'species'}), how='left')
        # Indexing
        barcode = file.split('adj_align_output/')[1].split("_all")[0]
        tax_profile["subject"] = barcode2subject_sample[barcode][0]
        tax_profile["is_vegan"] = barcode2subject_sample[barcode][1]
        tax_profile = tax_profile.set_index([tax_profile.subject, tax_profile.is_vegan]).sort_index()
        
        tax_profiles = pd.concat([tax_profiles,
                                  tax_profile])
    # Clean up of dataframe
    tax_profiles["kingdom"] = tax_profiles["kingdom"].str.strip("k__")
    tax_profiles["phylum"] = tax_profiles["phylum"].str.strip("p__")
    tax_profiles["class"] = tax_profiles["class"].str.strip("c__")
    tax_profiles["order"] = tax_profiles["order"].str.strip("o__")
    tax_profiles["family"] = tax_profiles["family"].str.strip("f__")
    tax_profiles["genus"] = tax_profiles["genus"].str.strip("g__")
    tax_profiles["species"] = tax_profiles["species"].str.strip("s__")
    tax_profiles = tax_profiles.drop(columns=["clade_name"])
    tax_profiles = tax_profiles[["kingdom", 
                                 "phylum", 
                                 "class", 
                                 "order", 
                                 "family", 
                                 "genus", 
                                 "species", 
                                 "relative_abundance", 
                                 "NCBI_tax_id", 
                                 "additional_species"]]
    
    return tax_profiles.sort_index()

### Recreating the plot from Zimmer et al. 2012
<img src="../microbiota_tax_data/zimmer_species_abundance_plot.png" alt="Zimmer et al. 2012" width="400"/>

In [78]:
def recreate_zimmer(tax_profiles, subject):
    """recreate a comparison of taxa that have been routinely analysed by Zimmer et al. 2012
    
    Keyword arguments:
    tax_profiles -- A pandas datatax_profilesframe containing the taxonomic profile per barcode for all barcodes
    
    Returns:
    zimmer_subset -- A pandas dataframe containing a subset of taxa of interest of the taxonomic profile per barcode for all barcodes 
    dataframes containing only the taxa analyzed by zimmer et al. 2012 as values
    """
    tax_profiles_subject = tax_profiles[tax_profiles.index.get_level_values('subject') == subject]
    
    # Subset for the Zimmer et al. 2012 bar plot
    bacteroides_subset = tax_profiles_subject[(tax_profiles_subject['genus'] == "Bacteroides") & (tax_profiles_subject['species'].isnull())]
    bifidobacteria_subset = tax_profiles_subject[(tax_profiles_subject['genus'] == "Bifidobacterium") & (tax_profiles_subject['species'].isnull())]
    ecoli_subset = tax_profiles_subject[(tax_profiles_subject['species'] == "Escherichia_coli")]
    enterobacter_subset = tax_profiles_subject[(tax_profiles_subject['family'] == "Enterobacteriaceae") & (tax_profiles_subject['genus'].isnull())]

    # Other taxa Zimmer et al. 2012 deemed of interest
    clostridia_subset = tax_profiles_subject[(tax_profiles_subject['class'] == "Clostridia") & (tax_profiles_subject['order'].isnull())]
    
    zimmer_subset = pd.concat([bacteroides_subset,
                               bifidobacteria_subset,
                               ecoli_subset,
                               enterobacter_subset,
                               clostridia_subset]).sort_index()
    
    
    plot_zimmer(zimmer_subset, subject)
    
    return zimmer_subset

In [79]:
def plot_zimmer(zimmer_subset, subject):
    """Plot the zimmer subset per subject as a grouped barplot. Based on code by Kylie Keijzer
    
    Keyword arguments:
    zimmer_subset -- A pandas dataframe containing a subset of taxa of interest of the taxonomic profile per barcode for all barcodes 
    dataframes containing only the taxa analyzed by zimmer et al. 2012 as values
    subject -- A string consisting of the subject ID
    """
    if not zimmer_subset.empty:
        taxa_abundance = pd.DataFrame()
        # Subset for the Zimmer et al. 2012 bar plot
        taxa_abundance['Bacteroides'] = zimmer_subset[(zimmer_subset['genus'] == "Bacteroides") & 
                                                      (zimmer_subset['species'].isnull())]['relative_abundance']
        taxa_abundance['Bifidobacterium'] = zimmer_subset[(zimmer_subset['genus'] == "Bifidobacterium") & 
                                                          (zimmer_subset['species'].isnull())]['relative_abundance']
        taxa_abundance['Escherichia_coli'] = zimmer_subset[(zimmer_subset['species'] == "Escherichia_coli")]['relative_abundance']
        taxa_abundance['Enterobacteriaceae'] = zimmer_subset[(zimmer_subset['family'] == "Enterobacteriaceae") & 
                                                             (zimmer_subset['genus'].isnull())]['relative_abundance']
        # Other taxa Zimmer et al. 2012 deemed of interest
        taxa_abundance['Clostridia'] = zimmer_subset[(zimmer_subset['class'] == "Clostridia") & 
                                                     (zimmer_subset['order'].isnull())]['relative_abundance']
        
#         display(taxa_abundance)

        taxa = ['Bacteroides', 'Bifidobacterium', 'Escherichia_coli', 'Enterobacteriaceae', 'Clostridia']
        diet = ['control', 'vegan']

        # Taking the mean of a single data point just for bokeh to accept the format of the ColumnDataSource
        grouped = taxa_abundance.groupby(['is_vegan']).mean()


        # Handling if there are either no datapoints before or after the vegan intervention
        if False in grouped.index:
            control = grouped[taxa].loc[False]
        else:
            control = [0 for taxon in taxa]
        if True in grouped.index:
            vegan = grouped[taxa].loc[True]
        else:
            vegan = [0 for taxon in taxa]

        data = {'taxa': taxa,
                'control': control,
                'vegan': vegan,
                'None' : [0 for i in taxa]}

        source = ColumnDataSource(data=data)

        # Creating the figure
        p = figure(x_range=taxa, y_range=(0, 100), plot_height=500, plot_width=800,
           title=f"Subject {subject}: Abundance of specific taxa of interest",
           x_axis_label="Taxon", y_axis_label="Relative abundance (%)", 
           toolbar_location=None, tools="")

        p.vbar(x=dodge('taxa', -0.12, range=p.x_range), 
               top='control', name='control', width=0.2, 
               source=source, color="#abdfff", legend_label="Regular diet", line_color="#75cbff")

        # middle label
        p.vbar(x=dodge('taxa',  0,  range=p.x_range), top='None', width=0.1, source=source)

        p.vbar(x=dodge('taxa',  0.12,  range=p.x_range), 
               top='vegan', name='vegan', width=0.2, 
               source=source, color="#ceffc4", legend_label="Vegan diet", line_color="#8dff75")

        p.x_range.range_padding = 0.1
        p.xgrid.grid_line_color = None
        p.legend.location = "top_right"
        p.legend.orientation = "horizontal"

        show(p)
            

### Dumbbell plot

In [80]:
def plot_dumbbell(tax_profiles, subject):
    """Plot the zimmer subset per subject as a grouped barplot. Based on code by Kylie Keijzer
    
    Keyword arguments:
    zimmer_subset -- A pandas dataframe containing a subset of taxa of interest of the taxonomic profile per barcode for all barcodes 
    dataframes containing only the taxa analyzed by zimmer et al. 2012 as values
    subject -- A string consisting of the subject ID
    """
    if not tax_profiles.empty:
        taxa_abundance = pd.DataFrame()
        tax_profiles_subject = tax_profiles[tax_profiles.index.get_level_values('subject') == subject]
        
        taxa = list(tax_profiles_subject[(tax_profiles_subject['class'].notnull()) & 
                                         (tax_profiles_subject['order'].isnull())]['class'])
        for taxon in taxa:
            taxa_abundance[taxon] = tax_profiles_subject[(tax_profiles_subject['class'] == taxon) & 
                                                         (tax_profiles_subject['order'].isnull())]['relative_abundance']

#         display(taxa_abundance)
        diet = ['control', 'vegan']
        
        taxa = list(set(taxa))

        # Taking the mean of a single data point just for bokeh to accept the format of the ColumnDataSource
        grouped = taxa_abundance.groupby(['is_vegan']).mean()
        
        # Handling if there are either no datapoints before or after the vegan intervention
        if False in grouped.index:
            control = grouped[taxa].loc[False]
        else:
            control = [0 for taxon in taxa]
        if True in grouped.index:
            vegan = grouped[taxa].loc[True]
        else:
            vegan = [0 for taxon in taxa]

        data = {'taxa': taxa,
                'control': control,
                'vegan': vegan,
                'None' : [0 for i in taxa]}

        source = ColumnDataSource(data=data)

        # Creating the figure
        p = figure(x_range=taxa, y_axis_type="log", plot_height=500, plot_width=800,
           title=f"Subject {subject}: Abundance of all classes",
           x_axis_label="Taxon", y_axis_label="Relative abundance (%)", 
           toolbar_location=None, tools="")

        p.circle(x=dodge('taxa', 0, range=p.x_range), y='control', name='control',
                source=source, color="darkblue", legend_label="Regular diet", size=25)
        
        p.circle(x=dodge('taxa', 0, range=p.x_range), y='vegan', name='vegan',
                source=source, color="darkgreen", legend_label="Vegan diet", size=25)

        p.x_range.range_padding = 0.1
        p.xgrid.grid_line_color = None
        p.legend.location = "top_right"
        p.legend.orientation = "horizontal"
        p.xaxis.major_label_orientation = pi/4

        show(p)

In [81]:
def main():
    config = get_config()
    gfkb, tax_profiles = load_data_microbiota(config['microbiota_files_path'])
    
    subjects = ["A", "B", "C", "D", "E"]
    for subject in subjects:
        plot_dumbbell(tax_profiles, subject)
        zimmer_subset = recreate_zimmer(tax_profiles, subject)
    
    with open("tmp.html",'w') as f:
        f.write(tax_profiles.to_html(justify="left"))
    display(tax_profiles)
    
if __name__ == '__main__':
    main()

kingdom         phylum        class          order  \
subject is_vegan                                                        
A       False      UNKNOWN           None         None           None   
        False     Bacteria           None         None           None   
        False      Viruses           None         None           None   
        False     Bacteria  Bacteroidetes         None           None   
        False     Bacteria     Firmicutes         None           None   
...                    ...            ...          ...            ...   
E       True      Bacteria  Bacteroidetes  Bacteroidia  Bacteroidales   
        True      Bacteria  Bacteroidetes  Bacteroidia  Bacteroidales   
        True      Bacteria  Bacteroidetes  Bacteroidia  Bacteroidales   
        True      Bacteria  Bacteroidetes  Bacteroidia  Bacteroidales   
        True      Bacteria  Bacteroidetes  Bacteroidia  Bacteroidales   

                          family        genus               species  \
subject is_vegan                                                      
A       False               None         None                  None   
        False               None         None                  None   
        False               None         None                  None   
        False               None         None                  None   
        False               None         None                  None   
...                          ...          ...                   ...   
E       True      Bacteroidaceae         None                  None   
        True      Bacteroidaceae  Bacteroides                  None   
        True      Bacteroidaceae  Bacteroides   Bacteroides_vulgatu   
        True      Bacteroidaceae  Bacteroides  Bacteroides_stercori   
        True      Bacteroidaceae  Bacteroides  Bacteroides_uniformi   

                  relative_abundance                        NCBI_tax_id  \
subject is_vegan                                                          
A       False              31.676630                                 -1   
        False              67.231544                                  2   
        False               1.091828                              10239   
        False              39.149381                              2|976   
        False              13.882107                             2|1239   
...                              ...                                ...   
E       True               11.730278            2|976|200643|171549|815   
        True               11.730278        2|976|200643|171549|815|816   
        True                6.390993    2|976|200643|171549|815|816|821   
        True                4.298627  2|976|200643|171549|815|816|46506   
        True                1.040657    2|976|200643|171549|815|816|820   

                                                 additional_species  
subject is_vegan                                                     
A       False                                                   NaN  
        False                                                   NaN  
        False                                                   NaN  
        False                                                   NaN  
        False                                                   NaN  
...                                                             ...  
E       True                                                    NaN  
        True                                                    NaN  
        True      k__Bacteria|p__Bacteroidetes|c__Bacteroidia|o_...  
        True                                                    NaN  
        True      k__Bacteria|p__Bacteroidetes|c__Bacteroidia|o_...  

[425 rows x 10 columns]